In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv
/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv


In [2]:
import pandas as pd 
import numpy as np 
import spacy
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
data_train=pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='latin-1')
data_test=pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='latin-1')

In [4]:
data_train.shape,data_test.shape

((41157, 6), (3798, 6))

In [5]:
data_train.keys()

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [6]:
data_train=data_train.drop(["UserName","ScreenName","Location","TweetAt"],axis=1)

In [7]:
data_train.head(5)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [8]:
data_train.Sentiment.value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

### Unbalanced data


We notice that the data in this case in  unbalanced witch will affect the model significantly
so to solve the issue we will extract from all categories equal samples and make a new dataframe balanced 


In [9]:
df=data_train.copy()

In [10]:
df_Neutral=df[df["Sentiment"]=="Neutral"].sample(5481,random_state=100)
df_Positive=df[df["Sentiment"]=="Positive"].sample(5481,random_state=100)
df_Negative=df[df["Sentiment"]=="Negative"].sample(5481,random_state=100)
df_ExPositive=df[df["Sentiment"]=="Extremely Positive"].sample(5481,random_state=100)
df_ExNegative=df[df["Sentiment"]=="Extremely Negative"].sample(5481,random_state=100)

In [11]:
final_data=pd.concat([df_Neutral,df_Positive,df_Negative,df_ExNegative,df_ExPositive])

In [12]:
final_data["Sentiment"].value_counts()

Neutral               5481
Positive              5481
Negative              5481
Extremely Negative    5481
Extremely Positive    5481
Name: Sentiment, dtype: int64

# Preprocessing 

We remove all the emojies ans symbols from the tweets 

In [13]:
import re
def remove_all_emojies(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
    

In [14]:
final_data["Sentiment"]=final_data["Sentiment"].apply(remove_all_emojies)

In [15]:
nlp=spacy.load("en_core_web_sm")
def remove_stop_words(text):
    docs=nlp(text)
    none_stop_words=list()
    for token in docs:
        if token.is_stop or token.is_punct:
            continue
        none_stop_words.append(token.text)
    return " ".join(none_stop_words)

We remove all the stop words like is ,always ,usual

In [16]:
final_data["Sentiment"]=final_data["Sentiment"].apply(remove_stop_words)

In [17]:
final_data["Sentiment"]=final_data["Sentiment"].map({"Positive":1,"Negative":2,"Neutral":3,"Extremely Positive":4,"Extremely Negative":5})

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(final_data.OriginalTweet,final_data.Sentiment,test_size=0.2,random_state=10)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
from sklearn.metrics import classification_report 

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

converting the text data into vectors 

In [22]:
v=CountVectorizer()
X_train_dtm=v.fit_transform(X_train)
X_test_dtm=v.transform(X_test)

First of all we will try NaiveBayesClassifier 

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
nb=MultinomialNB()
nb.fit(X_train_dtm,y_train)
y_pred=nb.predict(X_test_dtm)

We notice that NaiveBayesClassifier did poor performance specially in the first class 

In [25]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       0.28      0.35      0.31       873
           2       0.29      0.39      0.33       785
           3       0.43      0.71      0.53       648
           4       0.73      0.54      0.62      1557
           5       0.78      0.51      0.62      1618

    accuracy                           0.50      5481
   macro avg       0.50      0.50      0.48      5481
weighted avg       0.57      0.50      0.52      5481



We will try logisticRegression and see its performance 

In [26]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(class_weight="balanced")
logreg.fit(X_train_dtm,y_train)
y_pred_class=logreg.predict(X_test_dtm)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
from sklearn import metrics

We see that the LogisticRegression did slightly better than the previous algorithms 

In [28]:
print("Accuracy",metrics.accuracy_score(y_test,y_pred_class))

Accuracy 0.6112023353402664
